### Import libraries

In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout, LSTM
from sklearn.metrics import mean_absolute_error
from tensorflow.keras import layers

from datetime import datetime
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

### Import data

In [2]:
crypto_df = pd.read_csv("../input/g-research-crypto-forecasting/train.csv") 

In [3]:
crypto_df.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,2,40.0,2376.5800,2399.5000,2357.1400,2374.5900,19.233005,2373.116392,-0.004218
1,1514764860,0,5.0,8.5300,8.5300,8.5300,8.5300,78.380000,8.530000,-0.014399
2,1514764860,1,229.0,13835.1940,14013.8000,13666.1100,13850.1760,31.550062,13827.062093,-0.014643
3,1514764860,5,32.0,7.6596,7.6596,7.6567,7.6576,6626.713370,7.657713,-0.013922
4,1514764860,7,5.0,25.9200,25.9200,25.8740,25.8770,121.087310,25.891363,-0.008264


In [5]:
asset_details = pd.read_csv('../input/g-research-crypto-forecasting/asset_details.csv')

In [6]:
asset_details

,Asset_ID,Weight,Asset_Name
0,2,2.397895,Bitcoin Cash
1,0,4.304065,Binance Coin
2,1,6.779922,Bitcoin
3,5,1.386294,EOS.IO
4,7,2.079442,Ethereum Classic
5,6,5.894403,Ethereum
6,9,2.397895,Litecoin
7,11,1.609438,Monero
8,13,1.791759,TRON
9,12,2.079442,Stellar


In [7]:
# Select Asset_ID = 6 for Ethereum
crypto_df = crypto_df[crypto_df["Asset_ID"]==6] 
crypto_df.info(show_counts =True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1956200 entries, 5 to 24236799
Data columns (total 10 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   timestamp  1956200 non-null  int64  
 1   Asset_ID   1956200 non-null  int64  
 2   Count      1956200 non-null  float64
 3   Open       1956200 non-null  float64
 4   High       1956200 non-null  float64
 5   Low        1956200 non-null  float64
 6   Close      1956200 non-null  float64
 7   Volume     1956200 non-null  float64
 8   VWAP       1956200 non-null  float64
 9   Target     1955860 non-null  float64
dtypes: float64(8), int64(2)
memory usage: 164.2 MB


In [8]:
df = crypto_df.copy()

### Preprocess data

In [9]:
# fill missing values 
df = df.reindex(range(df.index[0],df.index[-1]+60,60),method='pad')
df = df.fillna(0)

In [10]:
# rename column timestamp to Date 
df.rename({'timestamp': 'Date'}, axis=1, inplace=True)

# rename Close to Price
df.rename(columns={'Close': 'Price'}, inplace=True)

In [11]:
# timestamp conversion
df.Date = df.Date.apply(lambda d: datetime.fromtimestamp(int(d)).strftime('%Y-%m-%d'))

In [12]:
# set index
df.set_index('Date', inplace=True)

In [13]:
# Convert to date array
timesteps = df.index.to_numpy()
prices = df['Price'].to_numpy()

timesteps[:10], prices[:10]

(array(['2018-01-01', '2018-01-01', '2018-01-01', '2018-01-01',
        '2018-01-01', '2018-01-01', '2018-01-01', '2018-01-01',
        '2018-01-01', '2018-01-01'], dtype=object),
 array([738.5075, 735.09  , 734.8025, 731.82  , 732.9325, 732.3425,
        731.8225, 732.1325, 732.605 , 729.415 ]))

### Prepare data for N-BEATS algorithm

In [14]:
HORIZON = 1 
WINDOW_SIZE = 7 

In [15]:
# NBeatsBlock layer 
class NBeatsBlock(tf.keras.layers.Layer):
  def __init__(self,  
               input_size: int,
               theta_size: int,
               horizon: int,
               n_neurons: int,
               n_layers: int,
               **kwargs): 
    super().__init__(**kwargs)
    self.input_size = input_size
    self.theta_size = theta_size
    self.horizon = horizon
    self.n_neurons = n_neurons
    self.n_layers = n_layers

    # Block contains stack of 4 fully connected layers each has ReLU activation
    self.hidden = [tf.keras.layers.Dense(n_neurons, activation="relu") for _ in range(n_layers)]
    
    # Output of block is a theta layer with linear activation
    self.theta_layer = tf.keras.layers.Dense(theta_size, activation="linear", name="theta")

  def call(self, inputs): 
    x = inputs 
    for layer in self.hidden:  
      x = layer(x)
    theta = self.theta_layer(x) 
    # Output the backcast and forecast from theta
    backcast, forecast = theta[:, :self.input_size], theta[:, -self.horizon:]
    return backcast, forecast

In [16]:
# Set up dummy NBeatsBlock layer to represent inputs and outputs
dummy_nbeats_block_layer = NBeatsBlock(input_size=WINDOW_SIZE, 
                                       theta_size=WINDOW_SIZE+HORIZON, # backcast + forecast 
                                       horizon=HORIZON,
                                       n_neurons=128,
                                       n_layers=4)

In [18]:
# Create dummy inputs (have to be same size as input_size)
dummy_inputs = tf.expand_dims(tf.range(WINDOW_SIZE) + 1, axis=0)  
dummy_inputs

<tf.Tensor: shape=(1, 7), dtype=int32, numpy=array([[1, 2, 3, 4, 5, 6, 7]], dtype=int32)>

In [19]:
# Pass dummy inputs to dummy NBeatsBlock layer
backcast, forecast = dummy_nbeats_block_layer(dummy_inputs)

# These are the activation outputs of the theta layer 
print(f"Backcast: {tf.squeeze(backcast.numpy())}")
print(f"Forecast: {tf.squeeze(forecast.numpy())}")

Backcast: [-0.81315935 -0.2589938  -0.31873906 -0.7641418   0.2292656  -0.4159255
  0.32254526]
Forecast: -1.1589165925979614


In [20]:
# Add windowed columns
ethereum_prices_nbeats = df.copy()
for i in range(WINDOW_SIZE):
  ethereum_prices_nbeats[f"Price+{i+1}"] = ethereum_prices_nbeats["Price"].shift(periods=i+1)
ethereum_prices_nbeats.drop(['Asset_ID', 'Count', 'Open', 'High', 'Low', 'Volume', 'VWAP', 'Target'], axis=1, inplace=True)
ethereum_prices_nbeats.dropna().head()


,Price,Price+1,Price+2,Price+3,Price+4,Price+5,Price+6,Price+7
Date,,,,,,,,
2018-01-01,732.1325,731.8225,732.3425,732.9325,731.8200,734.8025,735.0900,738.5075
2018-01-01,732.6050,732.1325,731.8225,732.3425,732.9325,731.8200,734.8025,735.0900
2018-01-01,729.4150,732.6050,732.1325,731.8225,732.3425,732.9325,731.8200,734.8025
2018-01-01,731.3200,729.4150,732.6050,732.1325,731.8225,732.3425,732.9325,731.8200
2018-01-01,733.5625,731.3200,729.4150,732.6050,732.1325,731.8225,732.3425,732.9325


In [21]:
# Features and labels
X = ethereum_prices_nbeats.dropna().drop("Price", axis=1)
y = ethereum_prices_nbeats.dropna()["Price"]

# Train-test-split sets
split_size = int(len(X) * 0.8)
X_train, y_train = X[:split_size], y[:split_size]
X_test, y_test = X[split_size:], y[split_size:]
len(X_train), len(y_train), len(X_test), len(y_test)

(323152, 323152, 80789, 80789)

In [22]:
# Train and test arrays into Tensor Datasets
train_features_dataset = tf.data.Dataset.from_tensor_slices(X_train)
train_labels_dataset = tf.data.Dataset.from_tensor_slices(y_train)

test_features_dataset = tf.data.Dataset.from_tensor_slices(X_test)
test_labels_dataset = tf.data.Dataset.from_tensor_slices(y_test)

# Combine features & labels
train_dataset = tf.data.Dataset.zip((train_features_dataset, train_labels_dataset))
test_dataset = tf.data.Dataset.zip((test_features_dataset, test_labels_dataset))

# Batch and prefetch for optimal performance
BATCH_SIZE = 1024  
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

train_dataset, test_dataset

(<PrefetchDataset shapes: ((None, 7), (None,)), types: (tf.float64, tf.float64)>,
 <PrefetchDataset shapes: ((None, 7), (None,)), types: (tf.float64, tf.float64)>)

### Modeling: N-BEATS algorithm

In [23]:
# Values from N-BEATS  
N_EPOCHS = 1000  
N_NEURONS = 512  
N_LAYERS = 4
N_STACKS = 30

INPUT_SIZE = WINDOW_SIZE * HORIZON  
THETA_SIZE = INPUT_SIZE + HORIZON

INPUT_SIZE, THETA_SIZE

(7, 8)

In [24]:
%%time

tf.random.set_seed(42)

# Setup N-BEATS Block layer
nbeats_block_layer = NBeatsBlock(input_size=INPUT_SIZE,
                                 theta_size=THETA_SIZE,
                                 horizon=HORIZON,
                                 n_neurons=N_NEURONS,
                                 n_layers=N_LAYERS,
                                 name="InitialBlock")

# Create input to stacks
stack_input = layers.Input(shape=(INPUT_SIZE), name="stack_input")

# Create initial backcast and forecast input 
backcast, forecast = nbeats_block_layer(stack_input)

# Add in subtraction residual link  
residuals = layers.subtract([stack_input, backcast], name=f"subtract_00") 

# Create stacks of blocks
for i, _ in enumerate(range(N_STACKS-1)):  

  # Use the NBeatsBlock to calculate the backcast as well as block forecast
  backcast, block_forecast = NBeatsBlock(
      input_size=INPUT_SIZE,
      theta_size=THETA_SIZE,
      horizon=HORIZON,
      n_neurons=N_NEURONS,
      n_layers=N_LAYERS,
      name=f"NBeatsBlock_{i}"
  )(residuals) # pass it in residuals (the backcast)

  # Create the double residual stacking
  residuals = layers.subtract([residuals, backcast], name=f"subtract_{i}") 
  forecast = layers.add([forecast, block_forecast], name=f"add_{i}")


CPU times: user 1.05 s, sys: 6.72 ms, total: 1.05 s
Wall time: 1.06 s


In [26]:
%%time

tf.random.set_seed(42)

# Setup N-BEATS Block layer
nbeats_block_layer = NBeatsBlock(input_size=INPUT_SIZE,
                                 theta_size=THETA_SIZE,
                                 horizon=HORIZON,
                                 n_neurons=N_NEURONS,
                                 n_layers=N_LAYERS,
                                 name="InitialBlock")

# Create input to stacks
stack_input = layers.Input(shape=(INPUT_SIZE), name="stack_input")

# Create initial backcast and forecast input 
backcast, forecast = nbeats_block_layer(stack_input)

# Add in subtraction residual link  
residuals = layers.subtract([stack_input, backcast], name=f"subtract_00") 

# Create stacks of blocks
for i, _ in enumerate(range(N_STACKS-1)):  

  # Use the NBeatsBlock to calculate the backcast as well as block forecast
  backcast, block_forecast = NBeatsBlock(
      input_size=INPUT_SIZE,
      theta_size=THETA_SIZE,
      horizon=HORIZON,
      n_neurons=N_NEURONS,
      n_layers=N_LAYERS,
      name=f"NBeatsBlock_{i}"
  )(residuals) # pass it in residuals (the backcast)

  # Create the double residual stacking
  residuals = layers.subtract([residuals, backcast], name=f"subtract_{i}") 
  forecast = layers.add([forecast, block_forecast], name=f"add_{i}")

# Put the stack model 
N_BEATS_model_7 = tf.keras.Model(inputs=stack_input, 
                         outputs=forecast, 
                         name="MODEL_7_N-BEATS")

# Compile with MAE loss and Adam optimizer
N_BEATS_model_7.compile(loss="mae",
                optimizer=tf.keras.optimizers.Adam(0.001),
                metrics=["mae", "mse"])

# Fit the model with EarlyStopping and ReduceLROnPlateau callbacks
N_BEATS_model_7.fit(train_dataset,
            epochs=N_EPOCHS,
            validation_data=test_dataset,
            verbose=0,  
            # model checkpoint callbacks
            callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=200, restore_best_weights=True),
                       tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=100, verbose=1)])


Epoch 00312: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 00412: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
CPU times: user 58min 26s, sys: 1min 37s, total: 1h 4s
Wall time: 2h 4min 14s


In [27]:
# Evaluate N-BEATS model on the test dataset
N_BEATS_model_7.evaluate(test_dataset)

79/79 [==============================] - 1s 18ms/step - loss: 5.2686 - mae: 5.2686 - mse: 75.1790


[5.2685866355896, 5.2685866355896, 75.17900085449219]

### Prediction inference

In [28]:
def make_preds(model, input_data):
  """
  Uses model to make predictions on input_data.
  """
  forecast = model.predict(input_data)
  return tf.squeeze(forecast) 

In [29]:
# Predictions with N-BEATS model
N_BEATS_model_preds = make_preds(N_BEATS_model_7, test_dataset)
N_BEATS_model_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([1227.8615, 1232.6721, 1229.2328, 1228.983 , 1227.4988, 1229.1234,
       1228.9984, 1220.7559, 1227.7506, 1228.4017], dtype=float32)>

### Evaluation

In [30]:
def evaluate_preds(y_true, y_pred):
  # Make sure float32 (for metric calculations)
  y_true = tf.cast(y_true, dtype=tf.float32)
  y_pred = tf.cast(y_pred, dtype=tf.float32)

  # Calculate various metrics
  mae = tf.keras.metrics.mean_absolute_error(y_true, y_pred)
  mse = tf.keras.metrics.mean_squared_error(y_true, y_pred) # puts emphasis on outliers (all errors get squared)
  rmse = tf.sqrt(mse)
  mape = tf.keras.metrics.mean_absolute_percentage_error(y_true, y_pred)
  
  return {"mae": mae.numpy(),
          "mse": mse.numpy(),
          "rmse": rmse.numpy(),
          "mape": mape.numpy()}

In [31]:
# Evaluate N-BEATS model predictions
N_BEATS_model_results = evaluate_preds(y_true=y_test,
                                 y_pred=N_BEATS_model_preds)
N_BEATS_model_results

{'mae': 5.2685866, 'mse': 75.179, 'rmse': 8.670583, 'mape': 0.22008494}